In [1]:
import os

def check_subfolders_in_images(directory_path):
    """
    Checks the number of subfolders inside 'Images_001' for each main folder.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Number of subfolders inside 'Images_001' for each folder.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in folders:
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                print(f"Subfolders inside '{image_001_path}': {len(subfolders)}")
            else:
                print(f"No 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"Directory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
check_subfolders_in_images(directory_path)


Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image1a\Images_001': 671
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image2a\Images_001': 387
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image3a\Images_001': 352
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image4a\Images_001': 277
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image5a\Images_001': 22
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image6a\Images_001': 789


In [ ]:
import os
import numpy as np
from tqdm import tqdm

try:
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
except ImportError:
    print("TensorFlow is not installed. You can install it by running: pip install tensorflow")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    Stores all subfolders' images in a single array and prints its shape.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Shape of the array containing resized images for all subfolders.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                
                # Array to store all images for the current main folder
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    
                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
load_images_from_subfolders(directory_path)


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

# Custom PatchEncoder layer
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection = tf.keras.Sequential(
            [
                tf.keras.layers.Conv2D(2, (3, 3), strides=(1, 1), activation='relu'),
                tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
                tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
                tf.keras.layers.Dense(projection_dim),
            ]
        )
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Load model
model_path = r"E:\five_frame_model.h5"
custom_objects = {'PatchEncoder': PatchEncoder}

try:
    model = load_model(model_path, custom_objects=custom_objects)
    print("\nModel loaded successfully!")
except Exception as e:
    print(f"\nError loading model: {e}")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")

                # Make individual predictions for the current folder
                make_predictions(model, all_images_array, folder)
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

def make_predictions(model, data, folder_name):
    """
    Makes predictions on the given data and prints the count of each class for a specific folder.
    """
    print(f"\nMaking predictions for folder '{folder_name}'...")
    predictions = model.predict(data, batch_size=32, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    unique_classes, counts = np.unique(predicted_classes, return_counts=True)
    print(f"\nPredicted Class Distribution for '{folder_name}':")
    for cls, count in zip(unique_classes, counts):
        print(f"Class {cls}: {count}")

# Load images and make individual predictions
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
#load_images_from_subfolders(directory_path)



Model loaded successfully!


In [11]:
# Load images and make individual predictions
directory_path = r"G:\datasets\2.Veliger_Data\2.Samples 4 - 6\Samples 4 - 6\4_39 Below Lake Waco Shallow 100 Gal"
load_images_from_subfolders(directory_path)


Processing main folders:   0%|                                               | 0/6 [00:00<?, ?it/s]
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   0%| | 0/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   0%| | 1/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   0%| | 3/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   0%| | 4/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   1%| | 5/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   1%| | 6/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   1%| | 7/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   1%| | 8/988 [00:
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':   1%| | 9/988 [00:


Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  12%| | 123/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 124/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 125/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 127/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 129/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 131/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  13%|▏| 133/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  14%|▏| 135/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  14%|▏| 137/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  14%|▏| 138/988 [0


Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  27%|▎| 264/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  27%|▎| 266/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  27%|▎| 267/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  27%|▎| 268/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  27%|▎| 270/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  28%|▎| 272/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  28%|▎| 274/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  28%|▎| 276/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  28%|▎| 278/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  28%|▎| 280/988 [0


Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  41%|▍| 410/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  42%|▍| 412/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  42%|▍| 413/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  42%|▍| 415/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  42%|▍| 417/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  42%|▍| 419/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  43%|▍| 421/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  43%|▍| 424/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  43%|▍| 426/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  43%|▍| 428/988 [0


Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  56%|▌| 552/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  56%|▌| 554/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  56%|▌| 556/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  56%|▌| 557/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  56%|▌| 558/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  57%|▌| 559/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  57%|▌| 560/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  57%|▌| 562/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  57%|▌| 563/988 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image1a':  57%|▌| 564/988 [0


...


Subfolders inside 'G:\datasets\2.Veliger_Data\2.Samples 4 - 6\Samples 4 - 6\4_39 Below Lake Waco Shallow 100 Gal\4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image5a\Images_001': shape of images: (1292, 5, 28, 28, 3)

Making predictions for folder '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image5a'...
41/41 [==============================] - 1s 27ms/step


Processing main folders:  83%|███████████████████████████████▋      | 5/6 [09:30<01:54, 114.13s/it]


Predicted Class Distribution for '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image5a':
Class 0: 1287
Class 1: 5



Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 0/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 2/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 3/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 4/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 5/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 6/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   0%| | 7/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   1%| | 8/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   1%| | 9/1547 [00
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   1%| | 10/1547 [0

Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 90/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 91/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 92/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 93/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 94/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 95/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 96/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 98/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   6%| | 99/1547 [0
Processing subfolders in '4_39 Below Lake Waco Shallow 100 Gal 9-17-21 Image6a':   7%| | 101/1547 [
